In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=2

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=2


In [2]:
from magma.language_model import get_gptj
from magma.config import MultimodalConfig

In [3]:
from magma.datasets import ImgCptDataset, collate_fn
from magma.utils import get_tokenizer

In [4]:
ENCODER_OUT_DIMS = {
    "nfresnet50": 2048,
    'clip_RN50': 2048,
    "clip": 512,
    "clip_resnet": 2560,
    "clip_resnet_large": 3072,
    "clip_resnet_tiny": 3072,
}

In [5]:
import os 
os.environ["CUDA_VISIBLE_DEVICES"]="2"
DEVICE = 2 

In [6]:
x_attn_block = GatedCrossAttentionBlock(
            config=config.cross_attention_config,
            text_token_dim=lm.config.hidden_size,
            visual_token_dim=ENCODER_OUT_DIMS[config.encoder_name],
        ).to(DEVICE)

NameError: name 'GatedCrossAttentionBlock' is not defined

In [ ]:
x_attn_block.perceiver_pipe(latents, media_mask=media_mask)

In [7]:
x_attn_block(word_embeddings).shape

NameError: name 'x_attn_block' is not defined

In [77]:
media_mask

tensor([[1, 1, 1,  ..., 3, 3, 3]], device='cuda:2')

In [12]:



import torch
from torch import nn, tanh, einsum
from torchtyping import TensorType
from einops import rearrange, repeat
from einops_exts import rearrange_many
from magma.utils import get_world_info


# class CrossAttentionTransformerBlock(nn.Module):
#     def __init__(
#         self,
#         lm_block: nn.Module,
#         config: dict,
#         token_dim: int = 4096,
#         **kwargs
#     ):
#         super().__init__()
#         self.lm_block = lm_block
#         self.cross_x_block = GatedCrossAttentionBlock(
#             config, token_dim, **kwargs)
#         self.media_locations = None
#         self.visual_features = None

#     def forward(self, embs, **kwargs):
#         logits = self.cross_x_block(
#             embs, self.media_locations, self.visual_features)
#         out = self.lm_block(logits, use_cache=False, **kwargs)
#         return out


class FeedForward(nn.Module):
    def __init__(self, dim, mult=4):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, dim * mult),
            nn.GELU(),
            nn.Linear(dim * mult, dim),
        )

    def forward(self, x: TensorType["Batch", "Sequence", "Dim"]):
        return self.net(x)


class MaskedCrossAttention(nn.Module):
    def __init__(
        self,
        text_token_dim: int = 4096,
        visual_token_dim: int = 2048,
        num_heads: int = 8,
        n_latents: int = 64, 
        head_dim: int = 64, 
        device: str = 'cuda:2' 

    ):
        super().__init__()
        local_rank, rank, world_size = get_world_info()
    #     self.device = device = f'cuda:{local_rank}' if local_rank is not None else 'cuda' if torch.cuda.is_available(
    # ) else 'cpu'
        self.device = 0
        self.num_heads = num_heads
        self.temp = 1 / (head_dim ** -0.5)
        self.softmax = nn.Softmax(dim=-1)


        self.v_k_w = nn.Linear(
            visual_token_dim, head_dim * num_heads * 2, bias=False)
        self.q_w = nn.Linear(
            text_token_dim, head_dim * num_heads, bias=False)
        self.out = nn.Linear(head_dim *num_heads, text_token_dim, bias=False)

    def forward(self,
                latent: TensorType["Batch", "Sequence", "TokenDim"],
                y: TensorType["Batch", "Sequence Length", "TokenDim"],
                media_mask: TensorType["Batch", "Sequence Length"]
                ):
        visual_features = rearrange(latent, 'b t n d -> b (t n) d')
        
        k, v = self.v_k_w(visual_features).chunk(2, dim=-1)
        q = self.q_w(y)
        q, k, v = rearrange_many((q, k, v), 'b n (h d) -> b h n d', h=8)
        sim = einsum('... i d, ... j d -> ... i j', q, k)
        print('SIM SHAPE',sim.shape)
        media_time = torch.arange(3).to(self.device) + 1
        print('media_time',media_time.shape, media_time.device, media_mask.device)
        text_to_media_mask = rearrange(media_mask, 'b i -> b 1 i 1') == repeat(media_time, 'j -> 1 1 1 (j m)', m=64)
        print(text_to_media_mask.shape)
        sim = sim.masked_fill(~text_to_media_mask, -torch.finfo(sim.dtype).max)
        print(sim)
        logits = sim.softmax(dim=-1)
        print('logits',logits.shape)
        text_without_media_mask = media_mask == 0
        text_without_media_mask = rearrange(text_without_media_mask, 'b i -> b 1 i 1')
        logits = logits.masked_fill(text_without_media_mask, 0.)
        logits = einsum('... i j, ... j d -> ... i d', logits, v)
        logits = rearrange(logits, 'b h n d -> b n (h d)')
        y = self.out(logits)
        return y


class GatedCrossAttentionBlock(nn.Module):
    def __init__(self, config, text_token_dim, visual_token_dim):
        super().__init__()
        self.x_attn = MaskedCrossAttention(
            text_token_dim = text_token_dim,
            visual_token_dim=visual_token_dim ,
            n_latents = config['n_latents']
            )
        self.tanh1 = nn.Parameter(torch.tensor([0.]))
        self.ffw = FeedForward(dim=text_token_dim)
        self.tanh2 = nn.Parameter(torch.tensor([0.]))

    def perceiver_pipe(self, visual_features, media_mask):
        self.media_mask = media_mask
        self.visual_features = visual_features

    def forward(self, embs: TensorType["Batch", "Sequence Length", "TokenDim"]):
        print('EMBS',embs.shape)

        x_attn = self.x_attn(self.visual_features, embs, self.media_mask)
        attn_out = embs + tanh(self.tanh1) * x_attn
        x_ffw = attn_out + self.ffw(x_attn) * tanh(self.tanh2)

        return x_ffw


In [13]:
## CROSS ATTENTION
import torch
from torch import nn
from torchtyping import TensorType
from einops import rearrange
from einops_exts import rearrange_many


class PerceiverAttentionBlock(nn.Module):
    def __init__(
        self,
        token_dim,
        output_dim,
        num_heads=8,
    ):
        super().__init__()
        self.num_heads = num_heads
        self.temp = 1 / (output_dim ** -0.5)
        self.softmax = nn.Softmax(dim=-1)

        self.v_k_w = torch.nn.Linear(
            token_dim, output_dim * num_heads * 2, bias=False)
        self.q_w = torch.nn.Linear(
            token_dim, output_dim * num_heads, bias=False)
        self.out = nn.Linear(output_dim*num_heads, token_dim, bias=False)

    def forward(self, q: TensorType["Batch", "Number of Images", "OutputDim", "TokenDim"], k_v: TensorType["Batch", "Number of Images", "Sequence", "TokenDim"]):

        # (batch, n_images, sequence_length + output_dim, embedding_dim)
        k_v = torch.cat((k_v, q), dim=-2)
        # (batch, n_images, sequence_length + output_dim, embedding_dim)
        k, v = self.v_k_w(k_v).tensor_split(2, dim=-1)
        q = self.q_w(q)  # (batch, n_images, output_dim, embedding_dim)

        q, k, v = rearrange_many(
            (q, k, v), 'b n s (h d) -> b n h s d', h=self.num_heads)

        q = q * self.temp

        # k = Image Input Sequence Dimension / q = Output Dimension
        sm = torch.einsum("b n h q d,b n h k d->b n h q k", q, k)
        attn = sm.softmax(dim=-1)

        ff_input_per_head = torch.einsum(
            "b n h q k,b n h k d -> b n h q d", attn, v)
        ff_input = rearrange(
            ff_input_per_head, "b n h s d -> b n s (h d)", h=self.num_heads)
        return self.out(ff_input)


class PerceiverResampler(nn.Module):
    def __init__(
        self,
        token_dim,
        num_layers:  int = 2,
        n_latents: int = 64,
        time: int = 1,
    ):
        super().__init__()
        self.register_parameter("learned_latents", nn.Parameter(
            torch.randn(n_latents, token_dim)))
        self.register_parameter('time_embeddings', nn.Parameter(
            torch.rand(time, 1, token_dim)))
        self.flatten = torch.nn.Flatten()
        self.perceiver_attention_layers = nn.ModuleList([])
        for _ in range(num_layers):
            self.perceiver_attention_layers.append(nn.ModuleList([
                PerceiverAttentionBlock(
                    token_dim=token_dim,  output_dim=n_latents),
                PerceiverFeedForwardLayer(token_dim=token_dim)
            ]))
        self.normalize = nn.LayerNorm(token_dim)

    def forward(self, x: TensorType["Batch", "Number of Images", "Time", "Sequence", "Token Dimesion"]):

        batch_size, number_of_images, sequence_length, token_dim = x.size()
        # x = rearrange(x, "b n s d -> b n (t s) d")
        latents = self.learned_latents.repeat(
            batch_size, number_of_images, 1, 1)
        x = x + self.time_embeddings[:number_of_images]

        for att_module, ff_layer in self.perceiver_attention_layers:
            latents = att_module(latents, x)
            latents = ff_layer(latents)

        return self.normalize(latents)


class PerceiverFeedForwardLayer(nn.Module):
    def __init__(
        self,
        token_dim: int = 3027,
        mult: int = 4,
    ):
        super().__init__()
        self.norm = nn.LayerNorm(token_dim)
        self.inner = nn.Linear(token_dim, mult*token_dim, bias=False)
        self.act = nn.GELU()
        self.outer = nn.Linear(mult*token_dim, token_dim, bias=False)

    def forward(self, x):
        return self.outer(self.act(self.inner(self.norm(x))))


In [14]:
import torch
import torch.nn as nn
from typing import Callable, Union
from torchtyping import patch_typeguard
from einops import rearrange
import timm
from activations.torch import Rational
import clip
from clip.model import Bottleneck
import copy

from functools import partial
from activations.utils.convert_network import convert_pytorch_model_to_rational
import os

# ----------------------------- Utils --------------------------------------

clip.model.LayerNorm = (
    nn.LayerNorm
)  # we need to patch this for clip to work with deepspeed
patch_typeguard()  # needed for torchtyping typechecks to work


class Lambda(torch.nn.Module):
    def __init__(self, fn: Callable):
        super().__init__()
        assert hasattr(fn, "__call__")
        self.fn = fn

    def forward(self, x):
        return self.fn(x)


# ------------------------- Image encoders ----------------------------------


def nfresnet50(
    device: Union[torch.device, str] = None, pretrained: bool = True, convert_to_rational: bool = False
) -> nn.Module:
    """
    Loads nfresnet50 model, removing the pooling layer and replacing it with
    an adaptive pooling layer.
    """
    encoder = torch.nn.Sequential(
        *list(timm.create_model("nf_resnet50", pretrained=pretrained).children())[:-1]
    )
    pooling = torch.nn.AdaptiveAvgPool2d((1, 1))
    encoder = torch.nn.Sequential(encoder, pooling)
    if device is not None:
        encoder = encoder.to(device)
    if convert_to_rational:
        encoder = convert_pytorch_model_to_rational(encoder)
    return encoder


def clip_encoder(
    device: Union[torch.device, str] = None, name: str = "clip", convert_to_rational: bool = False, approx_func: str = 'relu'
) -> nn.Module:
    """
    Loads clip's image encoder module, discarding the lm component.

    If the variant is a resnet model, we also remove the attention pooling.
    """
    if name in ["clip", "ViT-B/32"]:
        name = "ViT-B/32"
    elif name in ["clip_resnet", "RN50x4"]:
        name = "RN50x4"
    elif name in ['clip_RN50']:
        name = 'RN50'
    elif name in ["clip_resnet_large","clip_resnet_tiny", "RN50x16"]:
        name = "RN50x16"
    else:
        raise ValueError(f"encoder {name} not recognized")

    encoder = clip.load(name, device=device)[0].visual

    # if device is not None:
    #     encoder = encoder.to(device)

    if "RN" in name:
        # remove attention pooling
        encoder.attnpool = Lambda(
            partial(rearrange, pattern="b d h w -> b (h w) d")
        )  # remove attn pooling, just use reshaped features

    if convert_to_rational:
        encoder = convert_pytorch_model_to_rational(
            encoder, rational_cuda=device, approx_func="relu", submodule_class=Bottleneck)

    return encoder


def get_image_encoder(
    name: str, device: Union[torch.device, str] = None, pretrained: bool = False, convert_to_rational: bool = False
) -> torch.nn.Module:
    """
    Loads image encoder module
    """
    if name == "nfresnet50":
        encoder = nfresnet50(device=device, pretrained=pretrained,
                             convert_to_rational=convert_to_rational)
    elif "clip" in name:
        encoder = clip_encoder(device=device, name=name,
                               convert_to_rational=convert_to_rational)
    else:
        raise ValueError(f"image encoder {name} not recognized")
    return encoder

In [15]:
from torch import nn
def add_cross_attention_modules(lm, config):
    cross_attention_layers = []
    for l in range(len(lm.transformer.h)):
        layer_norm = getattr(lm.transformer.h[l], "ln_1")
        x_attn_block = GatedCrossAttentionBlock(
            config=config.cross_attention_config,
            text_token_dim=lm.config.hidden_size,
            visual_token_dim=ENCODER_OUT_DIMS[config.encoder_name],
        ).to('cuda')
        cross_attention_layers.append(l)
        setattr(lm.transformer.h[l], 'ln_1', nn.Sequential(
            *[x_attn_block, layer_norm]))
    return cross_attention_layers

In [16]:
from magma.transforms import get_transforms
from magma.image_encoders import get_image_encoder

enc = get_image_encoder('clip_resnet_large', convert_to_rational=False).to(f'cuda:{0}')
transforms = get_transforms(0, 'clip_resnet_large', enc.input_resolution)
config = MultimodalConfig.from_yml('/home/ml-mmeuer/adaptable_magma/fb20-dgx2-configs/dev.yml')
lm = get_gptj(config, from_pretrained=config.lm_name).to(f'cuda:{0}')
cross_attention_layers = add_cross_attention_modules(lm, config)
perceiver_resampler = PerceiverResampler(
                ENCODER_OUT_DIMS[config.encoder_name],
                n_latents=config.cross_attention_config['n_latents'],
            ).to(f'cuda:{0}')
seq_len = lm.config.max_position_embeddings
tokenizer = get_tokenizer(
            config.tokenizer_name, sequence_length=seq_len)

lm.resize_token_embeddings(len(tokenizer))
word_embedding = lm.transformer.wte

Loading GPTJ language model...
From EleutherAI/gpt-neo-125M


In [17]:
data = ImgCptDataset(config.train_dataset_dir, tokenizer=tokenizer, transforms=transforms, few_shot=3)
imgs = torch.stack((data[0][0], data[1][0], data[2][0]))
batch , n_imgs , c , h , w = imgs.size()
imgs = rearrange(imgs, 'b n c h w -> (b n) c h w')
captions =  torch.cat((data[0][1], data[1][1], data[2][1]))
imgs , captions = imgs.to('cuda:0'), captions.to('cuda:0')

enc_imgs = enc(imgs).to('cuda:0')
enc_imgs = rearrange(enc_imgs, '(b n) s d -> b n s d',  n=n_imgs)
latents = perceiver_resampler(enc_imgs)
# enc_imgs = rearrange(enc_imgs, '(b n) s d -> b n s d',  n=n_imgs)
emb_captions = word_embedding(captions)

In [18]:
media_pos = captions == tokenizer.cls_token_id
media_mask = media_pos.cumsum(dim=-1)

In [19]:
for l in cross_attention_layers:
    x_attn_block = getattr(
        lm.transformer.h[l], 'ln_1')[0]

    x_attn_block.perceiver_pipe(
        latents, media_mask=media_mask)

In [20]:
output = lm(inputs_embeds = emb_captions, output_hidden_states=False, )

EMBS torch.Size([3, 2048, 768])
SIM SHAPE torch.Size([3, 8, 2048, 192])
media_time torch.Size([3]) cuda:0 cuda:0
torch.Size([3, 1, 2048, 192])
tensor([[[[-5.4188e-01, -1.6184e+00, -3.3895e-01,  ..., -3.4028e+38,
           -3.4028e+38, -3.4028e+38],
          [-4.7702e-01, -5.5141e-01,  6.6971e-02,  ..., -3.4028e+38,
           -3.4028e+38, -3.4028e+38],
          [-4.2891e-01, -1.2530e+00,  1.1151e-01,  ..., -3.4028e+38,
           -3.4028e+38, -3.4028e+38],
          ...,
          [-3.4028e+38, -3.4028e+38, -3.4028e+38,  ...,  3.7321e-01,
            1.1395e-01,  1.1100e-01],
          [-3.4028e+38, -3.4028e+38, -3.4028e+38,  ...,  2.8427e-01,
            1.1162e-01,  1.1669e-01],
          [-3.4028e+38, -3.4028e+38, -3.4028e+38,  ...,  2.6279e-01,
            1.1931e-01,  1.0065e-01]],

         [[ 9.4439e-01,  9.5076e-01, -3.7520e-02,  ..., -3.4028e+38,
           -3.4028e+38, -3.4028e+38],
          [ 9.7041e-01,  1.3755e+00,  7.6440e-01,  ..., -3.4028e+38,
           -3.4028e+38

In [22]:
def build_labels(
    input_embeddings: TensorType["b", "n", "t", "s", "d"],
    captions: TensorType["b", "s"],
    eos_token,
) -> TensorType["b", "s"]:
    """
    Builds labels from input embeddings.

    Masks out the labels with -100 in positions up to the seq length of the embeddings, so loss is only computed for captions,
    and not for image tokens.
    Additionally, masks out everything *after* the first eos token.
    """
    shape = input_embeddings.shape[:2]  # b, s
    local_rank, rank, world_size = get_world_info()
    device = f'cuda:{local_rank}' if local_rank is not None else 'cuda' if torch.cuda.is_available(
    ) else 'cpu'

    assert captions.shape[1] >= shape[1]
    # make sure to add masked embedding tokens in the appropriate locations in the labels
    embedding_tokens = torch.zeros(
        shape, dtype=torch.int64).to(torch.device(device)) - 100
    labels = torch.cat(
        (embedding_tokens, captions[:, : -shape[1]]), dim=1
    )  # we truncate the sequence length of the captions, as they are always padded to the full sequence length

    # mask out repeating eos tokens
    for label in labels:
        for k, token in enumerate(label):
            if token == eos_token:
                label[k + 1:] = -100
                break

    return labels